In [23]:
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
import calendar
import datetime
api_key = 'b333a19b2c0397e8e4c1224b49b3e7cd'
api_base = 'http://ws.audioscrobbler.com/2.0/?format=json&'
user_name = 'dude0faw3'
data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key='\
                + api_key + '&user=' + user_name).json()

In [18]:
track_data = data['recenttracks']['track']
limit = data['recenttracks']['@attr']['totalPages']
page = 1
history = []
# for nested dictionary entry requests
class FakeNone:
    def get(*args):
        return FakeNone()
# ensures that there's no key error exception
def check_track(track):
    arr = [track.get('name', FakeNone()),
     track.get('artist', FakeNone()).get('#text', FakeNone()),
      track.get('date', FakeNone()).get('uts', FakeNone())
    ]
    return [None if type(x) == FakeNone else x for x in arr]

for track in track_data:
    history.append(check_track(track))
    
print('Starting! There are ' + limit + ' pages')
while int(data['recenttracks']['@attr']['page']) is not int(limit):
    if page % 25 is 0:
        print('Page %i' % page)
    page += 1
    data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key=' + api_key + \
                    '&user=' + user_name + '&page=' + str(page)).json()
    for track in data['recenttracks']['track']:
        history.append(check_track(track))

df = pd.DataFrame(history)
df.columns = ['track_name', 'artist', 'date']
df.tail()

Starting! There are 155 pages
Page 25
Page 50
Page 75
Page 100
Page 125
Page 150


,track_name,artist,date
30823,This Is What It Feels Like,Armin van Buuren,1407180582
30824,Ecstasy,ATB,1407180320
30825,Spaceman - Drown The Fish Remix,Hardwell,1407179792
30826,Legacy,Eminem,1407179496
30827,Spaceman - Drown The Fish Remix,Hardwell,1407179059


## Goals:
- most listened to songs/ artists (as well as during a time)
- songs/artists you've neglected
- most/least popular songs 

In [19]:
# determine the n-most frequently played songs from a given pandas series
def determine_frequencies(series, num_songs, neglected=False):
    names = series.values
    counter = Counter(names)
    return counter.most_common(num_songs)

determine_frequencies(df['track_name'], 10)

[('Playground [feat. Bugle & Arama]', 144),
 ('Cracks - Flux Pavilion Remix', 133),
 ('Give Me Everything', 129),
 ('The Hills', 127),
 ('Dum Dee Dum', 121),
 ('Legacy', 119),
 ('Alone', 119),
 ('Freeway - Flux Pavilion and Kill The Noise Remix', 111),
 ('Five Hours', 104),
 ("'Till I Collapse", 102)]

In [30]:
class TimeMachine:
    
    def __init__(self, year):
        self.seasons = {
            'spring': ( time.strptime('21 Mar ' + year, '%d %b %Y'),
                      time.strptime('21 Jun ' + year, '%d %b %Y')),
            'summer': ( time.strptime('21 Jun ' + year, '%d %b %Y'),
                      time.strptime('23 Sep ' + year, '%d %b %Y')),
            'fall': ( time.strptime('23 Sep ' + year, '%d %b %Y'),
                      time.strptime('21 Dec ' + year, '%d %b %Y')),
            'winter': ( time.strptime('21 Dec ' + year, '%d %b %Y'),
                        time.strptime('21 Mar ' + year, '%d %b %Y'))
        }
        self.year = year
        
    def set_year(self, new_year):
        self = self.__init__(new_year)
        
    def in_season(self, season, df):
        timeslice = self.seasons[season]
        def check_time(seconds):
            listen_time =  time.gmtime(int(seconds))
            return listen_time > timeslice[0] and listen_time < timeslice[1]
        arr_time_check = np.vectorize(check_time)
        return df[arr_time_check(df['date'])]
    
    def in_year(self, df):
        date = datetime.date(int(self.year), 1, 1)
        def check_time(seconds):
            listen_time =  datetime.date.fromtimestamp(int(seconds))
            return listen_time.year == date.year
        arr_time_check = np.vectorize(check_time)
        return df[ arr_time_check(df['date'])]
    
# make a timemachine to 2014, check out top artists from then
obj = TimeMachine('2014')
df_2015 = obj.in_year(df)
determine_frequencies(df_2015['artist'], 20)

[('Calvin Harris', 150),
 ('Blackmill', 149),
 ('Eminem', 143),
 ('Lindsey Stirling', 107),
 ('Bassnectar', 94),
 ('OneRepublic', 89),
 ('The Glitch Mob', 78),
 ('PANTyRAiD', 75),
 ('Doctor P', 68),
 ('Flux Pavilion', 68),
 ('The Script', 64),
 ('Maroon 5', 63),
 ('Linkin Park', 63),
 ('Nero', 60),
 ('Hardwell', 60),
 ('Gemini', 56),
 ('Lifehouse', 53),
 ('Major Lazer', 50),
 ('Zedd', 49),
 ('Armin van Buuren', 46)]